# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

#create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

print("Total number of rows: ",len(full_data_rows_list))

# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


Total number of rows:  8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Create and Upsert tables in Apache Cassandra

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

For the table we need artist, song_title, song_length, session_id and item_in_session.
A composite primary key of (session_id, item_in_session) is used as neither session_id nor item_in_session is unique. Also because we wish to include both columns in the where clause.

In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_table_query_1 = """
    CREATE TABLE IF NOT EXISTS songs_hist_by_session (
        session_id int,
        item_in_session int,
        song_title text,
        artist text,
        song_length float,
        PRIMARY KEY (session_id, item_in_session)
    );
"""
try:
    session.execute(create_table_query_1)
except Exception as e:
    print(e) 
                    

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """INSERT INTO songs_hist_by_session 
                    (session_id, item_in_session, song_title, artist, song_length) 
                    VALUES (%s, %s, %s, %s, %s) """ 
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[9], line[0], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """
    SELECT artist, song_title, song_length 
    FROM songs_hist_by_session 
    WHERE session_id=%s AND item_in_session = %s
"""
try:
    rows = session.execute(query, (338, 4))
except Exception as e:
    print(e)
    
for row in rows:
    print ("Artist:", row.artist, ", Song Title:", row.song_title, ", Song length:", row.song_length)

Artist: Faithless , Song Title: Music Matters (Mark Knight Dub) , Song length: 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

For the table we need user_id, session_id, item_in_session, artist text, song_title, user_firstname and user_lastname.
A primary key of (user_id, session_id) and (session_id, item_in_session, item_in_session) as composite primary key were used. The data is partitioned by user_id, session_id and clustered by item_in_session

In [11]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_table_query_2 = """
    CREATE TABLE IF NOT EXISTS song_by_user_session (
        user_id int,
        session_id int,
        item_in_session int,
        artist text,
        song_title text,
        user_firstname text,
        user_lastname text,
        PRIMARY KEY ((user_id, session_id),item_in_session)
    );
"""
try:
    session.execute(create_table_query_2)
except Exception as e:
    print(e) 
          

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """INSERT INTO song_by_user_session 
                    (user_id, session_id, item_in_session, artist,song_title, user_firstname,user_lastname) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s) """ 
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]),int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
                    
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """
    SELECT artist, song_title, user_firstname, user_lastname
    FROM song_by_user_session 
    WHERE user_id =%s AND session_id = %s
"""
try:
    rows = session.execute(query, (10,182))
except Exception as e:
    print(e)
    
for row in rows:
    print ("Artist:", row.artist, ", Song Title:", row.song_title, ", User Firstname:", row.user_firstname,", User Lastname:", row.user_lastname)

Artist: Down To The Bone , Song Title: Keep On Keepin' On , User Firstname: Sylvie , User Lastname: Cruz
Artist: Three Drives , Song Title: Greece 2000 , User Firstname: Sylvie , User Lastname: Cruz
Artist: Sebastien Tellier , Song Title: Kilometer , User Firstname: Sylvie , User Lastname: Cruz
Artist: Lonnie Gordon , Song Title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , User Firstname: Sylvie , User Lastname: Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

For the table we need user_id, song_title, user_firstname and user_lastname.
A primary key of (song_title,user_id) was used as neither song_title nor user_id is unique in the table. As we want to know who listened to what song, the data is partitioned by song_title andclustered by user_id.

In [12]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

create_table_query_3 = """
    CREATE TABLE IF NOT EXISTS song_by_listener_history (
        user_id int,
        song_title text,
        user_firstname text,
        user_lastname text,
        PRIMARY KEY (song_title,user_id)
    );
"""
try:
    session.execute(create_table_query_3)
except Exception as e:
    print(e) 
          

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """INSERT INTO song_by_listener_history 
                    (user_id,song_title, user_firstname,user_lastname) 
                    VALUES (%s, %s, %s, %s) """ 
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]),line[9], line[1],line[4]))
                    
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """
    SELECT  user_firstname, user_lastname
    FROM song_by_listener_history
    WHERE song_title = %s
"""
try:
    rows = session.execute(query, ("All Hands Against His Own",))
except Exception as e:
    print(e)
    
for row in rows:
    print ("User Firstname:", row.user_firstname,", User Lastname:", row.user_lastname)

                    

User Firstname: Jacqueline , User Lastname: Lynch
User Firstname: Tegan , User Lastname: Levine
User Firstname: Sara , User Lastname: Johnson


### Drop the tables before closing out the sessions

In [13]:
session.execute("DROP TABLE IF EXISTS songs_hist")
session.execute("DROP TABLE IF EXISTS song_by_session")
session.execute("DROP TABLE IF EXISTS song_listener_history")

### Close the session and cluster connection¶

In [14]:
session.shutdown()
cluster.shutdown()